In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
%config InlineBackend.figure_format = 'svg'

np.random.seed(3) # uncomment to get exactly the results from the slides

In [ ]:
def plot_sin_cos(suffix, observations, sin_x, sin_y, cos_x, cos_y):
    """
    Create a sin/cos plot as we want them
    """
    fig = plt.figure(figsize=(8, 4))
    fig.add_subplot(1, 2, 1)
    plt.title('sin(x)' + suffix)
    plt.ylim([-3, 3])
    plt.plot(input_range, observations['sin'], 'b.', alpha=0.75)
    if sin_x is not None:
        plt.plot(sin_x, sin_y, 'r-', linewidth=2)

    fig.add_subplot(1, 2, 2)
    plt.title('cos(x)' + suffix)
    plt.ylim([-3, 3])
    plt.plot(input_range, observations['cos'], 'b.', alpha=0.75)
    if cos_x is not None:
        plt.plot(cos_x, cos_y, 'r-', linewidth=2)

    plt.show()

## Generate the Data

In [ ]:
n_samples = 25

input_range = np.linspace(0, 2*np.pi, n_samples, dtype=np.float)
observations = pd.DataFrame()
observations['sin'] = (np.sin(input_range)
        + np.random.normal(0, 0.5, n_samples))
observations['cos'] = (np.cos(input_range)
        + np.random.normal(0, 0.25, n_samples))

## Scatterplot

In [ ]:
plot_sin_cos(' + noise', observations, None, None, None, None)

## LOESS

In [ ]:
smoothed_sin = lowess(observations['sin'], input_range, frac=0.4)
smoothed_cos = lowess(observations['cos'], input_range, frac=0.3)

In [ ]:
plot_sin_cos(': LOESS', observations, smoothed_sin[:, 0], smoothed_sin[:, 1], smoothed_cos[:, 0], smoothed_cos[:, 1])

## Kalman Filtering: no predictions

In [ ]:
from pykalman import KalmanFilter
dim = 2
initial_value_guess = observations.iloc[0]
observation_covariance = np.diag([0.5, 0.25]) ** 2

In [ ]:
kf = KalmanFilter(
    initial_state_mean=initial_value_guess,
    initial_state_covariance=observation_covariance,
    observation_covariance=observation_covariance
)
pred_state, state_cov = kf.smooth(observations)

In [ ]:
plot_sin_cos(': attempt 1', observations, input_range, pred_state[:, 0], input_range, pred_state[:, 1])

## Kalman Filtering: good prediction

In [ ]:
delta_t = np.pi * 2 / n_samples
transition_matrix = [[1, delta_t], [-delta_t, 1]]
transition_covariance = np.diag([0.2, 0.2]) ** 2
kf = KalmanFilter(
    initial_state_mean=initial_value_guess,
    initial_state_covariance=observation_covariance,
    observation_covariance=observation_covariance,
    transition_covariance=transition_covariance,
    transition_matrices=transition_matrix
)
pred_state, state_cov = kf.smooth(observations)


In [ ]:
plot_sin_cos(': Kalman filtered', observations, input_range, pred_state[:, 0], input_range, pred_state[:, 1])

In [ ]:
transition_matrix = [[1, -delta_t], [delta_t, 1]]
kf = KalmanFilter(
    initial_state_mean=initial_value_guess,
    initial_state_covariance=observation_covariance,
    observation_covariance=observation_covariance,
    transition_covariance=transition_covariance,
    transition_matrices=transition_matrix
)
pred_state, state_cov = kf.smooth(observations)

In [ ]:
plot_sin_cos(': Kalman fail', observations, input_range, pred_state[:, 0], input_range, pred_state[:, 1])